# Authoring Leaf Systems
notebook 실행 방법은 [index](./index.ipynb)를 참조하자.


In [2]:
import numpy as np
from pydrake.common.containers import namedview
from pydrake.common.value import Value
from pydrake.math import RigidTransform, RotationMatrix
from pydrake.systems.analysis import Simulator
from pydrake.systems.framework import BasicVector, LeafSystem
from pydrake.trajectories import PiecewisePolynomial

## Overview

[Modeling Dynamical Systems](./dynamical_systems.ipynb) 튜터리얼에서 기본 [LeafSystem](https://drake.mit.edu/doxygen_cxx/classdrake_1_1systems_1_1_leaf_system.html)을 작성해보면서 Drake 시스템 프레임워크에 대한 기본적인 지식을 얻도록 한다. 이 notebook에서 좀더 고급/완전한 시스템을 만드는데 대한 개요를 제공한다.

## Input and Output Ports

Leaf 시스템은 임의의 개수의 입력 및 출력 포트를 가질 수 있다. 포트에는 두 가지 기본 유형이 있습니다: 벡터 값(**vector-valued**) 포트와 추상 값(**abstract-valued**) 포트이다.

### Vector-valued Ports
벡터 값 포트부터 시작해보자. 이 포트는 사용하기가 가장 간단하다. 다음 시스템은 두 개의 벡터 입력 포트와 두 개의 벡터 출력 포트를 선언한다. 이 시스템은 두 개의 2-element 입력 벡터의 합과 차를 계산하여 결과를 두 개의 출력 포트에 놓습니다.

<table align=center cellpadding=0 cellspacing=0><tr align=center style="border:none;"><td style="vertical-align:middle; border:none;"><table cellspacing=0 cellpadding=0><tr><td align=right style="padding:5px 0px 5px 0px; border:none;">a &rarr;</td></tr><tr style="border:none;"><td align=right style="padding:5px 0px 5px 0px; border:none;">b &rarr;</td></tr></table></td><td align=center style="border:2px solid black;padding-left:20px;padding-right:20px;vertical-align:middle;" bgcolor=#F0F0F0>MyAdder</td><td style="vertical-align:middle; border:none;"><table cellspacing=0 cellpadding=0><tr><td align=left style="padding:5px 0px 5px 0px; border:none;">&rarr; sum</td></tr><tr><td align=left style="padding:5px 0px 5px 0px; border:none;">&rarr; difference</td></tr></table></td></tr></table>

In [3]:
class MyAdder(LeafSystem):
    def __init__(self):
        super().__init__()  # 반드시 base class를 초기화하자!
        self._a_port = self.DeclareVectorInputPort(name="a", size=2)
        self._b_port = self.DeclareVectorInputPort(name="b", size=2)
        self.DeclareVectorOutputPort(name="sum", size=2, calc=self.CalcSum)
        self.DeclareVectorOutputPort(name="difference",
                                     size=2,
                                     calc=self.CalcDifference)

    def CalcSum(self, context, output):
        # 2x1 벡터를 얻기 위해서 input ports를 evaluation한다.
        a = self._a_port.Eval(context)
        b = self._b_port.Eval(context)

        # 합을 출력 벡터에 쓴다.
        output.SetFromVector(a + b)

    def CalcDifference(self, context, output):
        # 2x1 벡터를 얻기 위해서 input ports를 evaluation한다.
        a = self._a_port.Eval(context)
        b = self._b_port.Eval(context)

        # 차를 출력 벡터에 쓴다.
        output.SetFromVector(a - b)

# 이 system의 instance와 context를 생성한다.
system = MyAdder()
context = system.CreateDefaultContext()

# 입력 포트를 고정 값으로 고정한다.
system.GetInputPort("a").FixValue(context, [3, 4])
system.GetInputPort("b").FixValue(context, [1, 2])

# 출력 포트를 evaluation한다.
print(f"sum: {system.GetOutputPort('sum').Eval(context)}")
print(f"difference: {system.GetOutputPort('difference').Eval(context)}")

sum: [4. 6.]
difference: [2. 2.]


몇 가지 명심해야할 사항들:
- 원하는 만큼의 입력 또는 출력을 선언할 수 있다(크기가 동일할 필요는 없음).
- Context를 허용하는 시스템 메서드 중 하나에서 Eval() 메서드를 통해 입력을 평가할 수 있다.
- 각 출력 포트에 대해 출력을 구현하는 다른 콜백 함수를 정의한다.
- 포트의 순서를 추적하는 것은 다소 번거로울 수 있다. 따라서 인덱스를 사용하거나 이름으로 참조하는 등 코드를 개선할 수 있다.

다음을 이해하고 있으면 도움이 된다:
- 시스템 메소드 구현에서 [InputPort::HasValue()](https://drake.mit.edu/doxygen_cxx/classdrake_1_1systems_1_1_input_port.html#a5536b94a4642fa4cf47164437dc66ae8)를 확인하여 입력을 선택 사항으로 만들 수도 있다.
- 입력 및 출력 포트는 자체적으로 타이밍 의미를 가지지 않는다. 출력 포트는 출력이 요청될 때마다 평가되며, 반복 계산을 피하기 위해 [캐싱](https://drake.mit.edu/doxygen_cxx/group__cache__design__notes.html)을 기본적으로 사용한다.
- `DeclareVectorOutputPort()` 및 `DeclareAbstractOutputPort()` 포트는 캐싱 시스템에서 출력 포트를 다시 evaluation해야 하는 시기를 결정하는 데 사용되는 옵션 인수 prerequisites_of_calc를 사용합니다. 좁은 전제 조건을 선언하면 코드 속도를 높일 수 있다. 놀랍게도, 전제 조건을 명시적으로 좁히면 일부 Diagram의 생성 속도를 크게 높일 수도 있다. 왜냐하면 이러한 사항이 지정되지 않은 경우 DiagramBuilder는 시스템을 기호 형식으로 변환하여 대수 루프를 확인하려고 시도하기 때문이다.
- The `DeclareVectorOutputPort()` and `DeclareAbstractOutputPort()` ports take an optional `prerequisites_of_calc` argument which is used by the caching system to determine when the output port needs to be re-evaluated. Declaring narrow prerequisites can speed up your code. Surprisingly, explicitly narrowing the prerequisites can also dramatically speed up the construction of some `Diagram`s, because when these are not specified the `DiagramBuilder` attempts to check for algebraic loops by converting your systems into symbolic form.


### Abstract-valued Ports
모든 입력과 출력을 벡터로 표현하는 것이 가장 좋은 방법은 아니다. Drake 시스템 프레임워크에서는 구조화된 타입(structured types)을 포트를 통해 전달하는 기능도 제공하는데, 이는 C++에서 "타입 소거(type erasure)"라는 기술을 사용하여 구현된다. 시스템 프레임워크의 관점에서 볼 때, 클래스들은 "추상적인" 데이터 타입을 다루며, 시스템 구현 자체만이 구조화된 타입을 다루는 방법을 알면 된다. 실제로 약간의 보일러플레이트(반복적인 코드)만 무시할 수 있다면 사용법은 간단하다. 입력 및 출력 포트에서 `RigidTransform`을 사용하는 `LeafSystem`의 예를 살펴보자. :

<table align=center cellpadding=0 cellspacing=0><tr align=center><td style="vertical-align:middle; border:none;"><table cellspacing=0 cellpadding=0><tr><td align=right style="padding:5px 0px 5px 0px; border:none;">in &rarr;</td></tr></table></td><td align=center style="border:2px solid black;padding-left:20px;padding-right:20px;vertical-align:middle" bgcolor=#F0F0F0>RotateAboutZ</td><td style="vertical-align:middle; border:none;"><table cellspacing=0 cellpadding=0><tr><td align=left style="padding:5px 0px 5px 0px; border:none;">&rarr; out</td></tr></table></td></tr></table>

In [ ]:
class RotateAboutZ(LeafSystem):
    def __init__(self):
        super().__init__()  # base class를 반드시 초기화!
        self.DeclareAbstractInputPort(name="in",
                                      model_value=Value(RigidTransform()))
        self.DeclareAbstractOutputPort(
            name="out",
            alloc=lambda: Value(RigidTransform()),
            calc=self.CalcOutput)

    def CalcOutput(self, context, output):
        # RigidTransform를 얻기 위해 입력 포트를 evaluation하기
        X_1 = system.get_input_port().Eval(context)

        X_2 = RigidTransform(RotationMatrix.MakeZRotation(np.pi / 2)) @ X_1

        # Set the output RigidTransform.
        output.set_value(X_2)

# Construct an instance of this system and a context.
system = RotateAboutZ()
context = system.CreateDefaultContext()

# Fix the input port to a constant value.
system.get_input_port().FixValue(context, RigidTransform())

# Evaluate the output port.
print(f"output: {system.get_output_port(0).Eval(context)}")

## State Variables

Systems store their state in the `Context`, and work with state in a very similar way to input and output ports. Again, we can have vector-valued state or abstract-valued state. Abstract state can only be updated in a discrete-time fashion, but vector-valued state can be declared to be either **discrete** or **continuous**.  

A discrete state $x_d$ will evolve based on update events, the most common of which would be a simple periodic event to define a difference equation.  We saw an example of this in the [introductory notebook](./dynamical_systems.ipynb).

In [ ]:
class SimpleDiscreteTimeSystem(LeafSystem):
    def __init__(self):
        super().__init__()

        state_index = self.DeclareDiscreteState(1)  # One state variable.
        self.DeclareStateOutputPort("y", state_index)  # One output: y=x.
        self.DeclarePeriodicDiscreteUpdateEvent(
            period_sec=1.0,  # One second time step.
            offset_sec=0.0,  # The first event is at time zero.
            update=self.Update) # Call the Update method defined below.

    # x[n+1] = x^3[n].
    def Update(self, context, discrete_state):
        x = context.get_discrete_state_vector().GetAtIndex(0)
        x_next = x**3
        discrete_state.get_mutable_vector().SetAtIndex(0, x_next)

# Instantiate the System.
system = SimpleDiscreteTimeSystem()
simulator = Simulator(system)
context = simulator.get_mutable_context()

# Set the initial conditions: x[0] = [0.9].
context.get_mutable_discrete_state_vector().SetFromVector([0.9])

# Run the simulation.
simulator.AdvanceTo(4.0)
print(context.get_discrete_state_vector())

You can find details on the implementation and order of these events [here](https://drake.mit.edu/doxygen_cxx/group__discrete__systems.html). It's also possible to define more complicated [events](https://drake.mit.edu/doxygen_cxx/group__events__description.html) for updating the discrete variables. 

Note that the states of a system are not immediately accessible to other systems in a `Diagram`. For a system to share its state, it must do that using an output port. We provide the `DeclareStateOutputPort()` method to make this common case easy.

The evolution of continuous vector-valued state variables is governed by a differential equation. While it is possible and convenient to declare many different groups of discrete state variables (they may even be updated at different rates by different events), we only define zero or one continuous state vector for a system, and define its dynamics by overloading the `LeafSystem::DoCalcTimeDerivatives()` method. Here is the simple continuous-time example in the [introductory notebook](./dynamical_systems.ipynb) again: 

In [ ]:
# Define the system.
class SimpleContinuousTimeSystem(LeafSystem):
    def __init__(self):
        super().__init__()

        state_index = self.DeclareContinuousState(1)  # One state variable.
        self.DeclareStateOutputPort("y", state_index)  # One output: y=x.

    # xdot(t) = -x(t) + x^3(t).
    def DoCalcTimeDerivatives(self, context, derivatives):
        x = context.get_continuous_state_vector().GetAtIndex(0)
        xdot = -x + x**3
        derivatives.get_mutable_vector().SetAtIndex(0, xdot)

# Instantiate the System.
system = SimpleContinuousTimeSystem()
simulator = Simulator(system)
context = simulator.get_mutable_context()

# Set the initial conditions: x(0) = [0.9]
context.SetContinuousState([0.9])

# Run the simulation.
simulator.AdvanceTo(4.0)
print(context.get_continuous_state_vector())

For abstract-valued state, we have a similar workflow. We declare the abstract state in the constructor, and define update events to update that state.  Discrete update events are restricted to updating *only* the discrete states, so we use "unrestricted" events to update the abstract state. Here is an example which stores a simple `PiecewisePolynomial` trajectory as abstract state:

In [ ]:
class AbstractStateSystem(LeafSystem):
    def __init__(self):
        super().__init__()

        self._traj_index = self.DeclareAbstractState(
            Value(PiecewisePolynomial()))
        self.DeclarePeriodicUnrestrictedUpdateEvent(period_sec=1.0,
                                                    offset_sec=0.0,
                                                    update=self.Update)

    def Update(self, context, state):
        t = context.get_time()
        traj = PiecewisePolynomial.FirstOrderHold(
            [t, t + 1],
            np.array([[-np.pi / 2.0 + 1., -np.pi / 2.0 - 1.], [-2., 2.]]))
        # Update the state
        state.get_mutable_abstract_state(int(self._traj_index)).set_value(traj)



system = AbstractStateSystem()
simulator = Simulator(system)
context = simulator.get_mutable_context()

# Set an initial condition for the abstract state.
context.SetAbstractState(0, PiecewisePolynomial())

# Run the simulation.
simulator.AdvanceTo(4.0)
traj = context.get_abstract_state(0).get_value()
print(f"breaks: {traj.get_segment_times()}")
print(f"traj({context.get_time()}) = {traj.value(context.get_time())}")

## Parameters

Parameters are like state, except that they are constant through the lifetime of a simulation. Parameters in the systems framework are declared and accessed almost identically to state, but they are never updated. Once again, we have vector-valued (declared with `DeclareNumericParameter`) and abstract-valued (via `DeclareAbstractParameter`) parameters.

<table align=center cellpadding=0 cellspacing=0><tr align=center><td style="vertical-align:middle; border:none;"><table cellspacing=0 cellpadding=0></table></td><td align=center style="border:2px solid black;padding-left:20px;padding-right:20px;vertical-align:middle" bgcolor=#F0F0F0>SystemWithParameters</td><td style="vertical-align:middle; border:none;"><table cellspacing=0 cellpadding=0><tr><td align=left style="padding:5px 0px 5px 0px; border:none;">&rarr; numeric</td></tr><tr><td align=left style="padding:5px 0px 5px 0px; border:none;">&rarr; abstract</td></tr></table></td></tr></table>

In [ ]:
class SystemWithParameters(LeafSystem):
    def __init__(self):
        super().__init__()  # Don't forget to initialize the base class.

        self.DeclareNumericParameter(BasicVector([1.2, 3.4]))
        self.DeclareAbstractParameter(
            Value(RigidTransform(RotationMatrix.MakeXRotation(np.pi / 6))))

        # Declare output ports to demonstrate how to access the parameters in
        # system methods.
        self.DeclareVectorOutputPort(name="numeric",
                                     size=2,
                                     calc=self.OutputNumeric)
        self.DeclareAbstractOutputPort(
            name="abstract",
            alloc=lambda: Value(RigidTransform()),
            calc=self.OutputAbstract)

    def OutputNumeric(self, context, output):
        output.SetFromVector(context.get_numeric_parameter(0).get_value())

    def OutputAbstract(self, context, output):
        output.set_value(context.get_abstract_parameter(0).get_value())

# Construct an instance of this system and a context.
system = SystemWithParameters()
context = system.CreateDefaultContext()

# Evaluate the output ports.
print(f"numeric: {system.get_output_port(0).Eval(context)}")
print(f"abstract: {system.get_output_port(1).Eval(context)}")

## Systems can "publish"

In addition to methods that update the state, and methods that evaluate an output port, another supported method in a `LeafSystem` is a callback to "publish". A "publish" method cannot modify any state: they are useful for broadcasting data outside of the systems framework (e.g. via a message-passing protocol like ROS), for terminating a simulation, for detecting errors, and for forcing boundaries between integration steps. They are declared in much the same way as other system callbacks.

In [ ]:
class MyPublishingSystem(LeafSystem):
    def __init__(self):
        super().__init__()

        # Calling `ForcePublish()` will trigger the callback.
        self.DeclareForcedPublishEvent(self.Publish)

        # Publish once every second.
        self.DeclarePeriodicPublishEvent(period_sec=1,
                                         offset_sec=0,
                                         publish=self.Publish)
        
    def Publish(self, context):
        print(f"Publish() called at time={context.get_time()}")

system = MyPublishingSystem()
simulator = Simulator(system)
simulator.AdvanceTo(5.3)

# We can also "force" a publish at a arbitrary time.
print("\ncalling ForcedPublish:")
system.ForcedPublish(simulator.get_context())


## Naming Vector Values

Abstract-valued ports, state, and/or parameters can be used to work with structured data. But it can also be convenient to use string names to reference the individual elements of vector-valued data. In Python, we recommend using a [`namedview`](https://drake.mit.edu/pydrake/pydrake.common.containers.html?highlight=namedview#pydrake.common.containers.namedview) workflow, which was inspired by Python's [`namedtuple`](https://docs.python.org/3.6/library/collections.html?highlight=namedtuple#collections.namedtuple) and is demonstrated in the example below. The Drake developers plan to [provide similar functionality in C++](https://github.com/RobotLocomotion/drake/issues/12566).

Note that the memory layout for a continuous state vector is slightly different than for discrete state, parameters, and input/output port data, so the syntax for dealing with it requires a slightly distinct `CopyToVector()` and `SetFromVector()`.  See [issue #9171](https://github.com/RobotLocomotion/drake/issues/9171).

Note that usages of `my_view[:]` are meant as a shorthand to get a NumPy array view of the `namedview` itself.
For more details, see the example and warning in the documentation for [`namedview`](https://drake.mit.edu/pydrake/pydrake.common.containers.html?highlight=namedview#pydrake.common.containers.namedview).

In [ ]:
# Define the system.
class NamedViewDemo(LeafSystem):
    MyDiscreteState = namedview("MyDiscreteState", ["a", "b"])
    MyContinuousState = namedview("MyContinuousState", ["x", "z", "theta"])
    MyOutput = namedview("MyOutput", ["x","a"])

    def __init__(self):
        super().__init__()

        self.DeclareDiscreteState(2)
        self.DeclarePeriodicDiscreteUpdateEvent(
            period_sec=1.0,
            offset_sec=0.0,
            update=self.DiscreteUpdate)
        self.DeclareContinuousState(3)
        self.DeclareVectorOutputPort(name="out", size=2, calc=self.CalcOutput)

    def DiscreteUpdate(self, context, discrete_values):
        discrete_state = self.MyDiscreteState(
            context.get_discrete_state_vector().value())
        continuous_state = self.MyContinuousState(
            context.get_continuous_state_vector().CopyToVector())
        next_state = self.MyDiscreteState(discrete_values.get_mutable_value())
        # Now we can compute the next state by referencing each element by name.
        next_state.a = discrete_state.a + 1
        next_state.b = discrete_state.b + continuous_state.x

    def DoCalcTimeDerivatives(self, context, derivatives):
        continuous_state = self.MyContinuousState(
            context.get_continuous_state_vector().CopyToVector())
        dstate_dt = self.MyContinuousState(continuous_state[:])
        dstate_dt.x = -continuous_state.x
        dstate_dt.z = -continuous_state.z
        dstate_dt.theta = -np.arctan2(continuous_state.z, continuous_state.x)
        derivatives.SetFromVector(dstate_dt[:])

    def CalcOutput(self, context, output):
        discrete_state = self.MyDiscreteState(
            context.get_discrete_state_vector().value())
        continuous_state = self.MyContinuousState(
            context.get_continuous_state_vector().CopyToVector())
        out = self.MyOutput(output.get_mutable_value())
        out.x = continuous_state.x
        out.a = discrete_state.a

# Instantiate the System.
system = NamedViewDemo()
simulator = Simulator(system)
context = simulator.get_mutable_context()

# Set the initial conditions.
initial_discrete_state = NamedViewDemo.MyDiscreteState([3, 4])
context.SetDiscreteState(initial_discrete_state[:])
initial_continuous_state = NamedViewDemo.MyContinuousState.Zero()
initial_continuous_state.x = 0.5
initial_continuous_state.z = 0.92
initial_continuous_state.theta = 0.23
context.SetContinuousState(initial_continuous_state[:])

# Run the simulation.
simulator.AdvanceTo(4.0)
print(
    NamedViewDemo.MyDiscreteState(context.get_discrete_state_vector().value()))
print(
    NamedViewDemo.MyContinuousState(
        context.get_continuous_state_vector().CopyToVector()))
print(NamedViewDemo.MyOutput(system.get_output_port().Eval(context)))


## Supporting Scalar Type Conversion (double, AutoDiff, and Symbolic)

In order to support AutoDiff and Symbolic types throughout the systems framework, LeafSystems can be written to support "scalar type conversion". In Python, adding this support requires a small amount of boilerplate.  Here is a simple example:

<table align=center cellpadding=0 cellspacing=0><tr align=center><td style="vertical-align:middle; border:none;"><table cellspacing=0 cellpadding=0><tr><td align=right style="padding:5px 0px 5px 0px; border:none;">state &rarr;</td></tr><tr><td align=right style="padding:5px 0px 5px 0px; border:none;">command &rarr;</td></tr></table></td><td align=center style="border:2px solid black;padding-left:20px;padding-right:20px;vertical-align:middle" bgcolor=#F0F0F0>RunningCost</td><td style="vertical-align:middle; border:none;"><table cellspacing=0 cellpadding=0><tr><td align=left style="padding:5px 0px 5px 0px; border:none;">&rarr; cost</td></tr></table></td></tr></table>

In [ ]:
from pydrake.systems.framework import LeafSystem_
from pydrake.systems.scalar_conversion import TemplateSystem
from pydrake.autodiffutils import AutoDiffXd
from pydrake.symbolic import Expression

@TemplateSystem.define("RunningCost_")
def RunningCost_(T):

    class Impl(LeafSystem_[T]):

        def _construct(self, converter=None, Q=np.eye(2)):
            super().__init__(converter)
            self._Q = Q
            self._state_port = self.DeclareVectorInputPort("state", 2)
            self._command_port = self.DeclareVectorInputPort("command", 1)
            self.DeclareVectorOutputPort("cost", 1, self.CostOutput)

        def _construct_copy(self, other, converter=None):
            # Any member fields (e.g. configuration values) need to be
            # transferred here from `other` to `self`.
            Impl._construct(self, converter=converter, Q=other._Q)

        def CostOutput(self, context, output):
            x = self._state_port.Eval(context)
            u = self._command_port.Eval(context)[0]
            output[0] = x.dot(self._Q.dot(x)) + u**2

    return Impl

RunningCost = RunningCost_[None]  # Default instantiation.

The important steps are:
- Add the `@TemplateSystem` decorator.
- Derive from `LeafSystem_[T]` instead of simply `LeafSystem`.
- Implement the `_construct` method *instead* of the typical `__init__` method.
- Implement the `_construct_copy` method, which needs to populate the same member fields as `_construct` (as we did with `self.Q` in this example).
- Add the default instantiation, so that you can continue to refer to the system as, e.g., `RunningCost` in addition to using `RunningCost_[float]`.

For further details, you can find the related documentation for scalar conversion in C++ [here](https://drake.mit.edu/doxygen_cxx/group__system__scalar__conversion.html) and the documentation for the [`@TemplateSystem`](https://drake.mit.edu/pydrake/pydrake.systems.scalar_conversion.html#pydrake.systems.scalar_conversion.TemplateSystem) decorator.

In [ ]:
# Having done this, we can still use the system in the original way:
system = RunningCost(Q=np.diag([10, 1]))
context = system.CreateDefaultContext()
system.get_input_port(0).FixValue(context, [1, 2])
system.get_input_port(1).FixValue(context, [3])
print(system.get_output_port().Eval(context))

# But we can also now use an autodiff or symbolic versions of the system,
# either by declaring them directly:
system_ad = RunningCost_[AutoDiffXd]()
system_symbolic = RunningCost_[Expression]()

# or by scalar conversion:
system_ad = system.ToAutoDiffXd()
system_symbolic = system.ToSymbolic()

# We can also convert the time, state, parameters, and (if needed) input ports
# from the original system:
context_symbolic = system_symbolic.CreateDefaultContext()
context_symbolic.SetTimeStateAndParametersFrom(context)
system_symbolic.FixInputPortsFrom(system, context, context_symbolic)
print(system_symbolic.get_output_port().Eval(context_symbolic))

## Further reading

[Caching in the systems framework](https://drake.mit.edu/doxygen_cxx/group__cache__design__notes.html)

[Declaring events](https://drake.mit.edu/doxygen_cxx/group__events__description.html)

[Stochastic systems](https://drake.mit.edu/doxygen_cxx/group__stochastic__systems.html)

Declaring [system constraints](https://drake.mit.edu/doxygen_cxx/classdrake_1_1systems_1_1_system_constraint.html): [equality](https://drake.mit.edu/doxygen_cxx/classdrake_1_1systems_1_1_leaf_system.html#a4948ad0241c67045b3c794874b2986a0) and [inequality](https://drake.mit.edu/doxygen_cxx/classdrake_1_1systems_1_1_leaf_system.html#a3bac306621c3f0839324649151c22af2).

[Writing continuous dynamics in implicit form](https://drake.mit.edu/doxygen_cxx/classdrake_1_1systems_1_1_system.html#a2bb4c1e3572a8009863b5a342fcb5c49)